In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [ ]:
df = load_latest_ray_experiment()
df.head()


In [ ]:
df["metric"] = df.apply(axis=1, func=lambda row: row["scheduler_metric"] if row["scheduler"] == "basic_scheduler" else "Simplex")

In [ ]:
px.line(
    df.sort_values("n_tasks"),
    x="n_tasks",
    y="n_allocated_tasks",
    color="metric",
    log_x=False,
    title="Number of allocated tasks (no profits)<br><sup>10 blocks, offline mixed-curves</sup>"
)